In [1]:
import json
import re
from neo4j.v1 import GraphDatabase, basic_auth

In [84]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth = basic_auth("neo4j", "temppassword1"))
session = driver.session()

In [2]:
cia = []
with open("../factbook.json") as infile:
    for line in infile:
        cia_fact = json.loads(line)
        cia.append(cia_fact)

In [87]:
for country in cia: #this will go through cia and plot everything
    b = country['region']
    if b == None:
        c = "None"
    else:
        b = b.replace(" ", "")
        c = re.sub('[^a-zA-Z0-9 \n\.]', '', b)
    
    name = country['countryname']
    code = country['countrycode']
    session.run("MERGE (a: "+c+" {name: {name}, code: {code}})", {"name": name, "code": code})
    
    try:  
        gdp = country['sections']['Economy'][2]['category_data'].split('(')[0] #i think some entry doesnt have brackets
        session.run("MATCH (a {name: {name}}) SET a.gdp = {gdp}", {"name": name, "gdp": gdp})
    except:
        print(" ")
    try:
        pop = country['sections']['People and Society'][4]['category_data'].split(' ')[0]
        session.run("MATCH (n {name: {name}}) SET n.population = {pop}", {"name": name, "pop": pop})
    except:
        print(" ")
    
    hasGov = 1
    govern
    try:
        govType = country['sections']['Government'][1]['category_data']
        govern = govType
        session.run("MERGE (b: GovType {type: {govType}})", {"govType": govType})
    except:
        hasGov = 0
    try:
        if(hasGov == 1):
            session.run("MATCH (a), (b)"
                       +"WHERE a.name = {name} AND b.type = {govern}"
                       +"MERGE (a)-[r:Goverment]->(b)",
                       {"name": name, "govern": govern})
    except:
        print(" ")
        
#     try:
#         dispute = country['sections']['Transnational Issues'][0]['category_data']
#         if dispute != "none":
#             session.run("MERGE (c: TransnationalIssue {type: InternatDispute})")
#             session.run("MATCH (a), (b)"
#                        +"WHERE a.name = {name} AND b.type = InternatDispute"
#                        +"MERGE (a)-[r:Is_Dispute {specific: {dispute}}]->(b)",
#                        {"name": name, "dispute": dispute})
#     except:
#         print(dispute)

    try:
        lang = country['sections']['People and Society'][2]['category_data']
        lang = "".join(lang)
        lang = lang.split(',')
        for a in lang:
            temLang = a.split(' ')
            tempLang = temLang[0]
            if temLang[0] == '':
                tempLang = temLang[1]
            session.run("MERGE (a: Language {type: {tempLang}})", {"tempLang": tempLang})
            session.run("MATCH (a), (b)"
                       +"WHERE a.name = {name} AND b.type = {tempLang}"
                       +"MERGE (a)-[r:Speaks]->(b)",
                       {"name": name, "tempLang": tempLang})
    except:
        print(" ")

In [3]:
print (cia[25])

{'sections': {'People and Society': [{'category_data': [{'sub_category_name': 'noun', 'sub_category_data': 'Belizean(s)'}, {'sub_category_name': 'adjective', 'sub_category_data': 'Belizean'}], 'category_name': 'Nationality', 'section_title': 'People and Society'}, {'category_data': 'mestizo 48.7%, Creole 24.9%, Maya 10.6%, Garifuna 6.1%, other 9.7% (2000 census)', 'category_name': 'Ethnic groups', 'section_title': 'People and Society'}, {'category_data': 'Spanish 46%, Creole 32.9%, Mayan dialects 8.9%, English 3.9% (official), Garifuna 3.4% (Carib), German 3.3%, other 1.4%, unknown 0.2% (2000 census)', 'category_name': 'Languages', 'section_title': 'People and Society'}, {'category_data': "Roman Catholic 39.3%, Pentacostal 8.3%, Seventh Day Adventist 5.3%, Anglican 4.5%, Mennonite 3.7%, Baptist 3.5%, Methodist 2.8%, Nazarene 2.8%, Jehovah's Witnesses 1.6%, other 9.9% (includes Baha'i Faith, Buddhism, Hinduism, Islam, and Mormon), other (unknown) 3.1%, none 15.2% (2010 census)", 'catego

In [11]:
print (cia[6]['sections']['Economy'][2]['category_data'].split('(')[0])

$12.8 billion 
